# Computer Vision, Lab 3: Segmentation

Today we'll experiment with methods to segment a scene. Since we've been working with a sample ground robot in an indoor environment, we'll take as an example the task of segmenting unoccupied ground plane space from obstacles.

If we can successfully determine which pixels in the image seen by our robot are on the flat floor and which are likely obstacles, we can combine that information with the ground plane homography-based rectification method we developed in the last lab to obtain a map of the space around the robot.

The same approach could be used by an autonomous vehicle to determine where, in the image from its front-facing camera, the road is and where possible obstacles are.

## Background: Color-based segmentation

In our indoor example, we have fairly consistent lighting and a consistently-colored floor, so a color based approach may work.

The task here is, given an input pixel at location $(x,y)$ with RGB color $(r,g,b)$, classify the pixel as "floor" or "not floor."

This is a classic machine learning problem. We have an input feature vector $(r, g, b)$ (some methods would also utilize $x$ and $y$ as well), and a desired output of 1
for floor and 0 for non-floor.

We probably want to transform the color space from RGB to HSV, since the RGB vector mixes color information with intensity information in the same measurements:

<img src="img/lab03-1.jpg" width="300"/>

whereas the HSV color space separates color from saturation ("color purity") and intensity:

<img src="img/lab03-2.jpg" width="300"/>

(Images are from Wikimedia Commons via the OpenCV documentation.)


Although we could use a fancy classifier like logistic regression or a SVM, this problem is easily solved with a generative model. We apply the principle of maximum a posteriori classification and Bayes' rule:

$$
f(h, s, v) = 
    \begin{cases}
        1 & p(\text{floor} \mid h, s, v) > 0.5 \\
        0 & \text{otherwise}
    \end{cases}
$$

$$
p(\text{floor} \mid h, s, v) = \frac{p(h, s, v \mid \text{floor}) p(\text{floor})}{p(h, s, v)}
$$

and since $p(h, s, v)$ is not known, we eliminate it:

$$
f(h, s, v) =
    \begin{cases}
        1 & p(h, s, v \mid \text{floor}) p(\text{floor}) > p(h, s, v | \text{not floor}) p(\text{not floor}) \\
        0 & \text{otherwise}
    \end{cases}
$$

The entity $p(\text{floor})$ is easy to estimate as the proportion of pixels in a sample of images that are floor pixels.

What about $p(h, s, v | floor)$?

The simplest approach here is just a lookup table. Since we're conditioning on floor, we just need to sample a bunch of floor pixels and record the frequency of each value of $h$, $s$, and $v$.

The problem with that approach is that we'd need something like $10 * 255 * 255 * 255$ pixels (168M!) to get a decent estimate of the value in every bin of this frequency table.

What we do instead is collapse some values of $h$, $s$, and $v$ into bins. For example, we might drop the 4 least significant bits of each of these values, then we'd end up needing only about $10 * 16 * 16 * 16$ (just 41K) samples to get a decent estimate.

Note that some methods would further drop the V element, which represents intensity, on the assumption that the global amount of lighting doesn't matter.

In fact, as the sun moves across the sky, the color composition of the light changes, and artificial light is quite different in color than sunlight. Still, it's a start.

So we want to collapse the $(h, s, v)$ vector into two features, $(h>>4, s>>4)$. We'll end up with a lookup table containing just $16*16 = 256$ elements.

## Create and save an image-plane-to-ground-plane homography to a YML file

Let's use a simple version of the program from lab 2 that calls `getPerspectiveTransform()`
to get a ground plane to image plane homography and save it to a YML file.

In [ ]:
#include <iostream>
#include <opencv2/opencv.hpp>

using namespace cv;
using namespace std;

#define VIDEO_FILE "robot.mp4"
#define HOMOGRAPHY_FILE "robot-homography.yml"

Mat matPauseScreen, matResult, matFinal;
Point point;
vector<Point> pts;
int var = 0;
int drag = 0;

// Create mouse handler function
void mouseHandler(int event, int x, int y, int, void*)
{
    if (var >= 4) return;
    if (event == EVENT_LBUTTONDOWN) // Left button down
    {
        drag = 1;
        matResult = matFinal.clone();
        point = Point(x, y);
        if (var >= 1) 
        {
            line(matResult, pts[var - 1], point, Scalar(0, 255, 0, 255), 2);
        }
        circle(matResult, point, 2, Scalar(0, 255, 0), -1, 8, 0);
        imshow("Source", matResult);
    }
    if (event == EVENT_LBUTTONUP && drag) // When Press mouse left up
    {
        drag = 0; var++;
        pts.push_back(point);
        matFinal = matResult.clone();
        if (var >= 4)
        {
            line(matFinal, pts[0], pts[3], Scalar(0, 255, 0, 255), 2);
            fillPoly(matFinal, pts, Scalar(0, 120, 0, 20), 8, 0);

            setMouseCallback("Source", NULL, NULL);
        }
        imshow("Source", matFinal);
    }
    if (drag)
    {
        matResult = matFinal.clone();
        point = Point(x, y);
        if (var >= 1) 
            line(matResult, pts[var - 1], point, Scalar(0, 255, 0, 255), 2);
        circle(matResult, point, 2, Scalar(0, 255, 0), -1, 8, 0);
        imshow("Source", matResult);
    }
}

int main()
{
    Mat matFrameCapture;
    Mat matFrameDisplay;
    int key = -1;

    VideoCapture videoCapture(VIDEO_FILE);
    if (!videoCapture.isOpened()) {
        cerr << "ERROR! Unable to open input video file " << VIDEO_FILE << endl;
        return -1;
    }

    while (key < 0)        // play video until press any key
    {
        // Get the next frame
        videoCapture.read(matFrameCapture);
        if (matFrameCapture.empty()) {
            // End of video file
            break;
        }
        float ratio = 640.0 / matFrameCapture.cols;
        resize(matFrameCapture, matFrameDisplay, cv::Size(), ratio, ratio, INTER_LINEAR);

        imshow(VIDEO_FILE, matFrameDisplay);
        key = waitKey(30);

        if (key >= 0)
        {
            destroyWindow(VIDEO_FILE);
            matPauseScreen = matFrameCapture;
            matFinal = matPauseScreen.clone();

            namedWindow("Source", WINDOW_AUTOSIZE);
            setMouseCallback("Source", mouseHandler, NULL);
            imshow("Source", matPauseScreen);
            waitKey(0);
            destroyWindow("Source");

            Point2f src[4];
            for (int i = 0; i < 4; i++)
            {
                src[i].x = pts[i].x * 1.0;
                src[i].y = pts[i].y * 1.0;
            }
            Point2f reals[4];
            reals[0] = Point2f(800.0, 800.0);
            reals[1] = Point2f(1000.0, 800.0);
            reals[2] = Point2f(1000.0, 1000.0);
            reals[3] = Point2f(800.0, 1000.0);

            Mat homography_matrix = getPerspectiveTransform(src, reals);
            std::cout << "Estimated Homography Matrix is:" << std::endl;
            std::cout << homography_matrix << std::endl;

            // perspective transform operation using transform matrix
            cv::warpPerspective(matPauseScreen, matResult, homography_matrix, matPauseScreen.size(), cv::INTER_LINEAR);
            imshow("Source", matPauseScreen);
            imshow("Result", matResult);

            waitKey(0);
        }
    }

    return 0;
}

### Python

In [ ]:
import cv2
import numpy as np

VIDEO_FILE = "robot.mp4"
HOMOGRAPHY_FILE = "robot-homography.yml"

matResult = None
matFinal = None
matPauseScreen = None

point = (-1, -1)
pts = []
var = 0 
drag = 0

# Mouse handler function has 5 parameters input (no matter what)
def mouseHandler(event, x, y, flags, param):
    global point, pts, var, drag, matFinal, matResult

    if (var >= 4):                           # if homography points are more than 4 points, do nothing
        return
    if (event == cv2.EVENT_LBUTTONDOWN):     # When Press mouse left down
        drag = 1                             # Set it that the mouse is in pressing down mode
        matResult = matFinal.copy()          # copy final image to draw image
        point = (x, y)                       # memorize current mouse position to point var
        if (var >= 1):                       # if the point has been added more than 1 points, draw a line
            cv2.line(matResult, pts[var - 1], point, (0, 255, 0, 255), 2)    # draw a green line with thickness 2
        cv2.circle(matResult, point, 2, (0, 255, 0), -1, 8, 0)             # draw a current green point
        cv2.imshow("Source", matResult)      # show the current drawing
    if (event == cv2.EVENT_LBUTTONUP and drag):  # When Press mouse left up
        drag = 0                             # no more mouse drag
        pts.append(point)                    # add the current point to pts
        var += 1                             # increase point number
        matFinal = matResult.copy()          # copy the current drawing image to final image
        if (var >= 4):                                                      # if the homograpy points are done
            cv2.line(matFinal, pts[0], pts[3], (0, 255, 0, 255), 2)   # draw the last line
            cv2.fillConvexPoly(matFinal, np.array(pts, 'int32'), (0, 120, 0, 20))        # draw polygon from points
        cv2.imshow("Source", matFinal);
    if (drag):                                    # if the mouse is dragging
        matResult = matFinal.copy()               # copy final images to draw image
        point = (x, y)                            # memorize current mouse position to point var
        if (var >= 1):                            # if the point has been added more than 1 points, draw a line
            cv2.line(matResult, pts[var - 1], point, (0, 255, 0, 255), 2)    # draw a green line with thickness 2
        cv2.circle(matResult, point, 2, (0, 255, 0), -1, 8, 0)         # draw a current green point
        cv2.imshow("Source", matResult)           # show the current drawing

def main():
    global matFinal, matResult, matPauseScreen
    key = -1;

    videoCapture = cv2.VideoCapture(VIDEO_FILE)
    if not videoCapture.isOpened():
        print("ERROR! Unable to open input video file ", VIDEO_FILE)
        return -1

    width  = videoCapture.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
    height = videoCapture.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float `height`

    # Capture loop 
    while (key < 0):
        # Get the next frame
        _, matFrameCapture = videoCapture.read()
        if matFrameCapture is None:
            # End of video file
            break

        ratio = 640.0 / width
        dim = (int(width * ratio), int(height * ratio))
        matFrameDisplay = cv2.resize(matFrameDisplay, dim)

        cv2.imshow(VIDEO_FILE, matFrameDisplay)
        key = cv2.waitKey(30)

        if (key >= 0):
            cv2.destroyWindow(VIDEO_FILE)
            matPauseScreen = matFrameCapture
            matFinal = matPauseScreen.copy()
            cv2.namedWindow("Source", cv2.WINDOW_AUTOSIZE)
            cv2.setMouseCallback("Source", mouseHandler)
            cv2.imshow("Source", matPauseScreen)
            cv2.waitKey(0)
            cv2.destroyWindow("Source")

            if (len(pts) < 4):
                return

            src = np.array(pts).astype(np.float32)
            reals = np.array([(800, 800),
                                (1000, 800),
                                (1000, 1000),
                                (800, 1000)], np.float32)

            homography_matrix = cv2.getPerspectiveTransform(src, reals);
            print("Estimated Homography Matrix is:")
            print(homography_matrix)

            h, w, ch = matPauseScreen.shape
            matResult = cv2.warpPerspective(matPauseScreen, homography_matrix, (w, h), cv2.INTER_LINEAR)
            matPauseScreen = cv2.resize(matPauseScreen, dim)
            cv2.imshow("Source", matPauseScreen)
            matResult = cv2.resize(matResult, dim)
            cv2.imshow("Result", matResult)

            cv2.waitKey(0)

main()

## Create a class for reading/writing homography data

Let's make a structure for our homography data. Objects
of class <code>HomographyData</code> will keep the information about a homography. It contains:

- cPoints: number of points for setting homography
- aPoints: points information
- matH: Homography matrix
- widthOut: image width of the output image
- heightOut: image height of the output image

We need two methods: reading and writing homographies.

At this step, you should create a new cpp file for create the special functions and structure. Please create 2 files: <code>HomographyData.h</code> and <code>HomographyData.cpp</code>

For Visual Studio users: <code>HomographyData.h</code> should be created in your
"Header Files" section, and <code>HomographyData.cpp</code> should be created in
your "Source Files" section.


### C++ header

Place the following in `HomographyData.h`:

In [ ]:
#include <opencv2/opencv.hpp>

class HomographyData
{
public:
    cv::Mat matH;
    int widthOut;
    int heightOut;
    int cPoints;
    cv::Point2f aPoints[4];

    HomographyData();
    HomographyData(std::string homography_file);

    bool read(std::string homography_file);
    bool write(std::string homography_file);
};

### C++ source file

Then provide the implementation in `HomographyData.cpp`:

In [ ]:
#include "homography.h"

Homography::Homography(std::string homography_file)
{
    read(homography_file);
}

HomographyData::HomographyData()
{
    cPoints = 0;
}

bool HomographyData::read(std::string homography_file)
{
    cv::FileStorage fileStorage(homography_file, cv::FileStorage::Mode::READ);
    if (!fileStorage.isOpened()) {
        return false;
    }
    cv::FileNode points = fileStorage["aPoints"];
    cv::FileNodeIterator it = points.begin(), it_end = points.end();
    cPoints = 0;
    for (int i = 0; it != it_end; it++, i++) {
        (*it) >> aPoints[i];
        cPoints++;
    }
    fileStorage["matH"] >> matH;
    fileStorage["widthOut"] >> widthOut;
    fileStorage["heightOut"] >> heightOut;
    fileStorage.release();
    return true;
}

bool HomographyData::write(std::string homography_file)
{
    cv::FileStorage fileStorage(homography_file, cv::FileStorage::Mode::WRITE);
    if (!fileStorage.isOpened()) {
        return false;
    }

    fileStorage << "aPoints" << "[";
    for (int i = 0; i < 4; i++)
    {
        fileStorage << aPoints[i];
    }
    fileStorage << "]";
    fileStorage << "matH" << matH;
    fileStorage << "widthOut" << widthOut;
    fileStorage << "heightOut" << heightOut;
    fileStorage.release();
    return true;
}

### Python

In [ ]:
import cv2
import numpy as np
from typing import List #use it for :List[...]

class Homograpy:
    matH = np.zeros((3, 3))
    widthOut : int
    heightOut : int
    cPoints : int
    aPoints:list = []

    def __init__(self, homography_file = None):
        self.cPoints = 0
        if homography_file is not None:
            self.read(homography_file)

    def read(self, homography_file):
        fileStorage = cv2.FileStorage(homography_file, cv2.FILE_STORAGE_READ)
        if not fileStorage.isOpened():
            return False

        self.cPoints = 0
        for i in range(points.size()):
            points = fileStorage.getNode("aPoints" + str(i))
            self.aPoints.append(points.mat())
            self.cPoints += 1
        self.matH = fileStorage.getNode("matH").mat()
        self.widthOut = int(fileStorage.getNode("widthOut").real())
        self.heightOut = int(fileStorage.getNode("heightOut").real())
        fileStorage.release()
        return True

    def write(self, homography_file):
        fileStorage = cv2.FileStorage(homography_file, cv2.FILE_STORAGE_WRITE)
        if not fileStorage.isOpened():
            return False

        for i in range(4):
            fileStorage.write("aPoints" + str(i), self.aPoints[i])

        fileStorage.write("matH", self.matH)
        fileStorage.write("widthOut", self.widthOut)
        fileStorage.write("heightOut", self.heightOut)
        fileStorage.release()
        return True

### Using it in C++

In [ ]:
#include "HomographyData.h"

...

// Write H to file

HomographyData homographyData;
for (int i = 0; i < 4; i++)
{
    homographyData.aPoints[i] = src[i];
    homographyData.cPoints++;
}
homographyData.matH = homography_matrix;
homographyData.widthOut = matPauseScreen.cols;
homographyData.heightOut = matPauseScreen.rows;
if (!homographyData.write(HOMOGRAPHY_FILE)) {
    cerr << "ERROR! Unable to write homography data file " << HOMOGRAPHY_FILE << endl;
    return -1;
}

...

// Read H from file

HomographyData homographyData(HOMOGRAPHY_FILE);
if (!homographyData.cPoints == 0) {
    cerr << "ERROR! Unable to read homography data file " << HOMOGRAPHY_FILE << endl;
    return -1;
}

### Using it in Python

In [ ]:
# Write H to file

homographyData = Homography()
homographyData.cPoints = 0
for i in range(4):
    homographyData.aPoints.append(src[i])
    homographyData.cPoints += 1
homographyData.matH = homography_matrix
homographyData.widthOut = width
homographyData.heightOut = height
homographyData.write(HOMOGRAPHY_FILE)

...

# Read H from file

homographyData = Homography(HOMOGRAPHY_FILE)
if homographyData.cPoints == 0:
    print("ERROR! Unable to read homography data file ", HOMOGRAPHY_FILE)
    return -1

### Results

The output file `robot-homography.yml` might contain the following. This is from the Python version; the C++ version is slightly
different.

    %YAML:1.0
    ---
    aPoints0: !!opencv-matrix
       rows: 2
       cols: 1
       dt: d
       data: [ 860., 49. ]
    aPoints1: !!opencv-matrix
       rows: 2
       cols: 1
       dt: d
       data: [ 1386., 52. ]
    aPoints2: !!opencv-matrix
       rows: 2
       cols: 1
       dt: d
       data: [ 1620., 375. ]
    aPoints3: !!opencv-matrix
       rows: 2
       cols: 1
       dt: d
       data: [ 784., 367. ]
    matH: !!opencv-matrix
       rows: 3
       cols: 3
       dt: d
       data: [ -1.1391186748382573e-03, -2.2794919900117339e-03,
           1.0627648167793186e-16, 1.6604445559011895e-05,
           -3.2917310423991194e-03, -3.1863931027743840e-02,
           -1.6241141452077307e-09, -1.8250980274059071e-06,
           -9.0126408686546570e-04 ]
    widthOut: 2419
    heightOut: 1250

## Use a pre-computed homography in a new program

### C++

In [ ]:
#include <opencv2/opencv.hpp>
#include <iostream>
#include "homography.h"

using namespace cv;
using namespace std;

#define VIDEO_FILE "robot.mp4"
#define HOMOGRAPHY_FILE "robot-homography.yml"

void displayFrame(cv::Mat& matFrameDisplay, int iFrame, int cFrames, tsHomographyData* pHomographyData) {
    for (int i = 0; i < pHomographyData->cPoints; i++) {
        cv::circle(matFrameDisplay, pHomographyData->aPoints[i], 10, cv::Scalar(255, 0, 0), 2, cv::LINE_8, 0);
    }
    imshow(VIDEO_FILE, matFrameDisplay);
    stringstream ss;
    ss << "Frame " << iFrame << "/" << cFrames;
    ss << ": hit <space> for next frame or 'q' to quit";
    //cv::displayOverlay(VIDEO_FILE, ss.str(), 0);  // for linux + qt
    putText(matFrameDisplay, ss.str(), Point(10, 30), FONT_HERSHEY_SIMPLEX, 1.0, Scalar(0, 0, 255), 3);
}

int main()
{
    cv::Mat matFrameCapture;
    cv::Mat matFrameDisplay;
    int cFrames;
    tsHomographyData homographyData;

    cv::VideoCapture videoCapture(VIDEO_FILE);
    if (!videoCapture.isOpened()) {
        cerr << "ERROR! Unable to open input video file " << VIDEO_FILE << endl;
        return -1;
    }
    cFrames = (int)videoCapture.get(cv::CAP_PROP_FRAME_COUNT);

    // Create a named window that will be used later to display each frame
    cv::namedWindow(VIDEO_FILE, (unsigned int)cv::WINDOW_NORMAL | cv::WINDOW_KEEPRATIO | cv::WINDOW_GUI_EXPANDED);

    // Read homography from file
    if (!readHomography(HOMOGRAPHY_FILE, &homographyData)) {
        cerr << "ERROR! Unable to read homography data file " << HOMOGRAPHY_FILE << endl;
        return -1;
    }

    int iFrame = 0;
    while (true) {

        // Block for next frame

        videoCapture.read(matFrameCapture);
        if (matFrameCapture.empty()) {
            // End of video file
            break;
        }

        displayFrame(matFrameCapture, iFrame, cFrames, &homographyData);

        int iKey;
        do {
            iKey = cv::waitKey(10);
            if (getWindowProperty(VIDEO_FILE, cv::WND_PROP_VISIBLE) == 0) {
                return 0;
            }
            if (iKey == (int)'q' || iKey == (int)'Q') {
                return 0;
            }
        } while (iKey != (int)' ');
        iFrame++;
    }

    return 0;
}

### Python

In [ ]:
import cv2
import numpy as np
from homography import tsHomographyData, readHomography, writeHomography

VIDEO_FILE = "robot.mp4"
HOMOGRAPHY_FILE = "robot-homography.yml"

def displayFrame(matFrameDisplay, iFrame, cFrames, pHomographyData):
    for i in range(pHomographyData.cPoints):
        cv2.circle(matFrameDisplay, pHomographyData.aPoints[i], 10, (255, 0, 0), 2, cv.LINE_8, 0)

    cv2.imshow(VIDEO_FILE, matFrameDisplay)
    ss = "Frame " + str(iFrame) + "/" + str(cFrames)
    ss += ": hit <space> for next frame or 'q' to quit";
    # cv2.displayOverlay(VIDEO_FILE, ss, 0);  # for linux + qt
    cv2.putText(matFrameDisplay, ss, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3);

def main():
    global matFinal, matResult, matPauseScreen
    key = -1;

    videoCapture = cv2.VideoCapture(VIDEO_FILE)
    if not videoCapture.isOpened():
        print("ERROR! Unable to open input video file ", VIDEO_FILE)
        return -1

    width  = videoCapture.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
    height = videoCapture.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float `height`

    cFrames = videoCapture.get(cv2.CAP_PROP_FRAME_COUNT)

    cv2.namedWindow(VIDEO_FILE, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_EXPANDED)

    homographyData = readHomography(HOMOGRAPHY_FILE)
    if homographyData is None:
        print("ERROR! Unable to read homography data file ", HOMOGRAPHY_FILE)
        return -1

    iFrame = 0
    # Capture loop 
    while (key < 0):
        # Get the next frame
        _, matFrameCapture = videoCapture.read()
        if matFrameCapture is None:
            # End of video file
            break

        matFrameCapture = videoCapture.read(matFrameCapture)
        if matFrameCapture is None:
            # End of video file
            break

        displayFrame(matFrameCapture, iFrame, cFrames, homographyData)

        iKey = -1
        while iKey != ord(' '):
            iKey = cv2.waitKey(10)
            if (getWindowProperty(VIDEO_FILE, cv2.WND_PROP_VISIBLE) == 0):
                return 0
            if (iKey == ord('q') or iKey == ord('Q')):
                return 0

        iFrame += 1

        return

main()

## Quick version of segmentation

As a simple experiment, find a pixel in the image that is clearly on the floor, then
convert the input image to the HSV colorspace using code

    cvtColor(matFrameBGR, matFrameHSV, COLOR_BGR2HSV);

If you display HSV as an image you might get something like this:

<img src="img/lab03-4.png" width="600"/>

Output the HSV values at your preferred location. Create a 2D array <code>double aProbFloorHS[16][16]</code>
and set the selected bin to a probability of 1.0 and all others to 0.
Show, in a second window, a mask for the pixels selected by $p(h, s, v \mid \text{floor}) > 0.5$.

## Obtain HS probabilities using a machine learning model

Next, take the first frame from the video, load it in <link>[gimp](https://www.gimp.org/downloads/)</link> or other image editing software, and make a mask for the floor pixels.

To extract frames from the video at a frame rate of one frame per second of the video, try at the command line

    $ sudo apt-get install ffmpeg
    $ ffmpeg -i robot.mp4 -r 1 -f image2 frame-%03d.png

Or you can save an image sequence from your OpenCV program:

    stringstream ss;
    ss << fixed << setprecision(3) << setfill('0');     // set fix digit prefix length to 3 and the blank digit is filled by 0
    ss << "frame-" << setw(3) << iFrame << ".png";      // set iFrame length to 3 from command above
    imwrite(ss.str(), matFrameCapture);

To make a mask in gimp, add a 50% transparent layer, color pixels you want to select, and delete the original image layer with: 

(right click $\rightarrow$ Layer $\rightarrow$ Stack $\rightarrow$ Select Next Layer), and then

(right click $\rightarrow$ Layer $\rightarrow$ Delete Layer).

<img src="img/lab03-5.png" width="300"/>

<img src="img/lab03-6.png" width="600"/>

<img src="img/lab03-7.png" width="600"/>

You can copy the resulting layer then create a new image from the clipboard and export as a black and white PNG.

<img src="img/lab03-8.png" width="600"/>

Add code to your program to read a mask along with the corresponding frame, apply the mask to the first frame of the video,
and accumulalte the entries in the <code>aProbFloorHS</code> array. You'll also need a <code>aProbNonFloorHS</code> array and
total pixel counts for each array.

## Independent exercise: Try to get a good probabilistic floor color model yourself

Finally, display the mask in a partially transparent color on top of the image as the video is rendered. How well does it work? You might want to add additional images to your training set. Consider a tool such as hasty.ai to mark up multiple images.

With a training set of 19 images, 64 bins for H, 16 bins for S, and 16 bins for V, I got a leave-one-out cross validated test accuracy of around 95%, with an F1 for the floor of 0.98 and an F1 for the obstacles of 0.51. See how well your model works and include this information in your report.

## The report

You should turn in, before the next lab, a brief report of your experiments and results. Also upload a video showing off your results.
